# Libraries

In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup

import numpy as np

# Data

In [2]:
# link of the wikipedia page
link = 'https://en.wikipedia.org/wiki/List_of_Indian_satellites'
# get all tables from the link
all_tables = pd.read_html(link)
# total no. of tables in the page
len(all_tables)

15

In [3]:
# first table
# all_tables[0]

In [4]:
# last table
# all_tables[5]

In [5]:
# container for all the tables
list_of_tables = []

# first 6 tables contains the data that we want
for table_no in range(6):
    
    # create a temporary table
    temp_table = all_tables[table_no]
#     print(temp_table.shape)
    
    # drop mutli index
    temp_table.columns = temp_table.columns.droplevel(0)
    temp_table.columns = temp_table.columns.droplevel(0)
#     print(temp_table.shape)
    
    # create a columns from previous rows
    for i in range(len(temp_table)):
        temp_table.loc[i+1, 'COSPAR ID'] = temp_table.loc[i, 'SatCat #']
        temp_table.loc[i+1, 'Launch Mass'] = temp_table.loc[i, 'Dry Mass']
        temp_table.loc[i+1, 'Note'] = temp_table.loc[i, 'Periapsis']

    # drop rows with even no. index (0, 2, 4 ...) (repeated columns)
    temp_table = temp_table.drop([i for i in range(len(temp_table)) if i%2==0])

    # replace wikipedia link no.s
    temp_table = temp_table.replace({'\[\d+\]' : ''}, regex=True)    
    
    # append temporary table to list_of_tables
    list_of_tables.append(temp_table)
    
#     print(temp_table.columns)
#     print('\n', '='*20, '\n')

In [6]:
# concatenate all the tables to form the full table
full_table = pd.concat(list_of_tables).reset_index(drop=True)

In [7]:
# first few rows
# full_table.head()

In [8]:
# names of columns
# full_table.columns

In [9]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   #                  122 non-null    object
 1   Name               122 non-null    object
 2   Discipline         122 non-null    object
 3   SatCat #           121 non-null    object
 4   Dry Mass           110 non-null    object
 5   On-board Power     117 non-null    object
 6   Launch Date        122 non-null    object
 7   Launch Vehicle     122 non-null    object
 8   Launch Site        122 non-null    object
 9   Periapsis          118 non-null    object
 10  Apoapsis           119 non-null    object
 11  Semi-Major Axis    117 non-null    object
 12  Period             119 non-null    object
 13  Inclination        119 non-null    object
 14  Longitude‡         120 non-null    object
 15  Eccentricity       112 non-null    object
 16  Epoch Start        120 non-null    object
 1

# Preprocessing

In [10]:
full_table = full_table.replace('Not Applicable', np.nan)
full_table = full_table.replace('Not applicable', np.nan)
full_table = full_table.replace('–', np.nan)

In [11]:
full_table = full_table.drop('Refs(ISRO portal)', axis = 1)

In [12]:
# rearrange columns
full_table = full_table[['#', 'Name', 'Discipline', 'COSPAR ID', 'SatCat #', 'Launch Mass', 'Dry Mass', 
                         'On-board Power', 'Launch Date', 'Launch Vehicle', 'Launch Site', 'Periapsis', 
                         'Apoapsis', 'Semi-Major Axis', 'Period', 'Inclination', 'Longitude‡', 
                         'Eccentricity', 'Epoch Start', 'Decay Date', 'Note']]

# rename columns
full_table.columns = ['#', 'Name', 'Discipline', 'COSPAR ID', 'SatCat #', 'Launch Mass', 'Dry Mass', 
                      'On-board Power', 'Launch Date', 'Launch Vehicle', 'Launch Site', 'Periapsis', 
                      'Apoapsis', 'Semi-Major Axis', 'Period', 'Inclination', 'Longitude', 
                      'Eccentricity', 'Epoch Start', 'Decay Date', 'Note']

full_table.head()

,#,Name,Discipline,COSPAR ID,SatCat #,Launch Mass,Dry Mass,On-board Power,Launch Date,Launch Vehicle,...,Periapsis,Apoapsis,Semi-Major Axis,Period,Inclination,Longitude,Eccentricity,Epoch Start,Decay Date,Note
0,1,Aryabhatta,Earth Sciences Space Physics,1975-033A,07752,360 kg (790 lb),NaN,46 W,"19 April 1975, 13:10:00 IST",Interkosmos-II,...,568 km (353 mi),611 km (380 mi),NaN,96.5 mins,50.7°,NaN,0.00308,"19 April 1975, 01:30:00 IST",11 February 1992,Active technological experience in building an...
1,2,Bhaskara Sega-I,Astronomy Communications Engineering Earth Sci...,1979-051A,11392,444 kg (979 lb),NaN,47 W,"7 June 1979, 16:00:00 IST",Modified SS-5 (SKean IRBM) plus Upper Stage,...,512 km (318 mi),557 km (346 mi),NaN,95.2 mins,50.7°,NaN,0.00325,"7 June 1979, 01:30:00 IST",17 February 1989,First experimental remote sensing satellite. C...
2,3,Rohini Technology Payload,Experimental,NaN,NaN,35 kg (77 lb),NaN,3 W,10 August 1979,SLV-3-E1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Intended for measuring in-flight performance o...
3,4,Rohini RS-1 (Rohini-1B),Earth Sciences,1980-062A,11899,35 kg (77 lb),NaN,16 W,"18 July 1980, 8:01:00 IST",SLV-3-E2,...,305 km (190 mi),919 km (571 mi),NaN,96.9 mins,44.7°,NaN,0.04389,"18 July 1980, 1:30:00 IST",20 May 1981,Used for measuring in-flight performance of se...
4,5,Rohini RS-D1 (Rohini-2),Earth Sciences,1981-051A,12491,38 kg (84 lb),NaN,16 W,"31 May 1981, 10:30:00 IST",SLV-3-D1,...,186 km (116 mi),418 km (260 mi),NaN,90.5 mins,46.3°,NaN,0.01735,"31 May 1981, 1:30:00 IST",8 June 1981,Used for conducting some remote sensing techno...


In [13]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   #                122 non-null    object
 1   Name             122 non-null    object
 2   Discipline       122 non-null    object
 3   COSPAR ID        112 non-null    object
 4   SatCat #         110 non-null    object
 5   Launch Mass      121 non-null    object
 6   Dry Mass         35 non-null     object
 7   On-board Power   82 non-null     object
 8   Launch Date      122 non-null    object
 9   Launch Vehicle   122 non-null    object
 10  Launch Site      122 non-null    object
 11  Periapsis        102 non-null    object
 12  Apoapsis         103 non-null    object
 13  Semi-Major Axis  90 non-null     object
 14  Period           102 non-null    object
 15  Inclination      103 non-null    object
 16  Longitude        48 non-null     object
 17  Eccentricity     28 non-null     ob

In [14]:
full_table.to_csv('raw.csv', index=False)

# Data Cleaning

In [15]:
full_table.columns

Index(['#', 'Name', 'Discipline', 'COSPAR ID', 'SatCat #', 'Launch Mass',
       'Dry Mass', 'On-board Power', 'Launch Date', 'Launch Vehicle',
       'Launch Site', 'Periapsis', 'Apoapsis', 'Semi-Major Axis', 'Period',
       'Inclination', 'Longitude', 'Eccentricity', 'Epoch Start', 'Decay Date',
       'Note'],
      dtype='object')

In [16]:
[
# '#' 
'Name' 
'Discipline' 
'COSPAR ID' 
'SatCat #' 
# 'Launch Mass'
# 'Dry Mass' 
# 'On-board Power' 
# 'Launch Date' 
'Launch Vehicle'
'Launch Site' 
# 'Periapsis' 
# 'Apoapsis' 
# 'Semi-Major Axis' 
'Period'
'Inclination' 
'Longitude' 
'Eccentricity' 
'Epoch Start' 
'Decay Date'
'Note'
]

['NameDisciplineCOSPAR IDSatCat #Launch VehicleLaunch SitePeriodInclinationLongitudeEccentricityEpoch StartDecay DateNote']

### Launch Mass

In [17]:
full_table['Launch Mass'][:5]

0    360 kg (790 lb)
1    444 kg (979 lb)
2      35 kg (77 lb)
3      35 kg (77 lb)
4      38 kg (84 lb)
Name: Launch Mass, dtype: object

In [18]:
# launch mass in kg

launch_mass_in_kg = full_table['Launch Mass'].str.extract('([0-9,.]*)')[0]
launch_mass_in_kg = launch_mass_in_kg.str.replace(',', '')
launch_mass_in_kg = pd.to_numeric(launch_mass_in_kg, errors='coerce')

launch_mass_in_kg[:5]

0    360.0
1    444.0
2     35.0
3     35.0
4     38.0
Name: 0, dtype: float64

In [19]:
full_table['Launch Mass'] = launch_mass_in_kg
full_table = full_table.rename(columns={"Launch Mass": "Launch Mass in kg"})

### Dry Mass

In [20]:
full_table['Dry Mass'][60:65]

60    1,426 kg (3,144 lb)
61      559 kg (1,232 lb)
62                    NaN
63                    NaN
64                    NaN
Name: Dry Mass, dtype: object

In [21]:
# dry mass in kg

dry_mass_in_kg = full_table['Dry Mass'].str.extract('([0-9,.]*)')[0]
dry_mass_in_kg = dry_mass_in_kg.str.replace(',', '')
dry_mass_in_kg = pd.to_numeric(dry_mass_in_kg, errors='coerce')

dry_mass_in_kg[60:65]

60    1426.0
61     559.0
62       NaN
63       NaN
64       NaN
Name: 0, dtype: float64

In [22]:
full_table['Dry Mass'] = dry_mass_in_kg
full_table = full_table.rename(columns={"Dry Mass": "Dry Mass in kg"})

### On-board Power

In [23]:
full_table['On-board Power'].head()

0    46 W
1    47 W
2     3 W
3    16 W
4    16 W
Name: On-board Power, dtype: object

In [24]:
on_board_power_in_W = full_table['On-board Power'].str.extract('(\d*)')[0]
on_board_power_in_W = pd.to_numeric(on_board_power_in_W, errors='coerce')

on_board_power_in_W.head()

0    46.0
1    47.0
2     3.0
3    16.0
4    16.0
Name: 0, dtype: float64

In [25]:
full_table['On-board Power'] = on_board_power_in_W
full_table = full_table.rename(columns={"On-board Power": "On-board Power in W"})

### Launch Date

In [26]:
full_table['Launch Date'].head()

0    19 April 1975, 13:10:00 IST
1      7 June 1979, 16:00:00 IST
2                 10 August 1979
3      18 July 1980, 8:01:00 IST
4      31 May 1981, 10:30:00 IST
Name: Launch Date, dtype: object

In [27]:
launch_date = full_table['Launch Date'].str.extract('(\d+ [A-Za-z]+ \d{4})')[0]
launch_date = pd.to_datetime(launch_date, errors='coerce')

launch_time = full_table['Launch Date'].str.extract('(\d{1,2}\:\d{1,2}(\:\d{1,2})?)')[0]

launch_time_zone = full_table['Launch Date'].str.extract('([A-Z]{3})')[0]

In [28]:
launch_date.head()

0   1975-04-19
1   1979-06-07
2   1979-08-10
3   1980-07-18
4   1981-05-31
Name: 0, dtype: datetime64[ns]

In [29]:
launch_time.head()

0    13:10:00
1    16:00:00
2         NaN
3     8:01:00
4    10:30:00
Name: 0, dtype: object

In [30]:
launch_time_zone.head()

0    IST
1    IST
2    NaN
3    IST
4    IST
Name: 0, dtype: object

In [31]:
full_table['Launch Date'] = launch_date
full_table['Launch Time'] = launch_time
full_table['Launch Timezone'] = launch_time_zone

### Periapsis, Apoapsis, Semi-Major Axis

In [32]:
full_table[['Periapsis', 'Apoapsis', 'Semi-Major Axis']][34:39]

,Periapsis,Apoapsis,Semi-Major Axis
34,"35,741.2 km (22,208.6 mi)","35,845.9 km (22,273.6 mi)","42,166 km (26,201 mi)"
35,"35,874.2 km (22,291.2 mi)","35,980.2 km (22,357.1 mi)","42,298 km (26,283 mi)"
36,"35,892.6 km (22,302.6 mi)","35,936.5 km (22,329.9 mi)","42,285 km (26,275 mi)"
37,"35,576.4 km (22,106.2 mi)","35,716.3 km (22,193.1 mi)","42,017 km (26,108 mi)"
38,824.2 km (512.1 mi),829.5 km (515.4 mi),"7,197 km (4,472 mi)"


In [33]:
periapsis_in_km = full_table['Periapsis'].str.extract('([0-9,.]*)')[0].str.replace(',', '')
periapsis_in_km = pd.to_numeric(periapsis_in_km, errors='coerce')

apoapsis_in_km = full_table['Apoapsis'].str.extract('([0-9,.]*)')[0].str.replace(',', '')
apoapsis_in_km = pd.to_numeric(apoapsis_in_km, errors='coerce')

semi_major_axis_in_km = full_table['Semi-Major Axis'].str.extract('([0-9,.]*)')[0].str.replace(',', '')
semi_major_axis_in_km = pd.to_numeric(semi_major_axis_in_km, errors='coerce')

In [34]:
periapsis_in_km[34:39]

34    35741.2
35    35874.2
36    35892.6
37    35576.4
38      824.2
Name: 0, dtype: float64

In [35]:
apoapsis_in_km[34:39]

34    35845.9
35    35980.2
36    35936.5
37    35716.3
38      829.5
Name: 0, dtype: float64

In [36]:
semi_major_axis_in_km[34:39]

34    42166.0
35    42298.0
36    42285.0
37    42017.0
38     7197.0
Name: 0, dtype: float64

In [37]:
full_table['Periapsis'] = periapsis_in_km
full_table['Apoapsis'] = apoapsis_in_km
full_table['Semi-Major Axis'] = semi_major_axis_in_km

full_table = full_table.rename(columns={"Periapsis": "Periapsis in km", 
                                        "Apoapsis": "Apoapsis in km", 
                                        "Semi-Major Axis": "Semi-Major Axis in km"})

### Period

In [38]:
full_table['Period'].head()

0    96.5 mins
1    95.2 mins
2          NaN
3    96.9 mins
4    90.5 mins
Name: Period, dtype: object

In [39]:
period = full_table['Period'].str.replace(' (mins|minutes).*', '')
period = period.str.replace(',', '')
period = pd.to_numeric(period, errors='coerce')
period.head()

0    96.5
1    95.2
2     NaN
3    96.9
4    90.5
Name: Period, dtype: float64

In [40]:
full_table['Period'] = period

### Inclination

In [41]:
full_table['Inclination'].head()

0    50.7°
1    50.7°
2      NaN
3    44.7°
4    46.3°
Name: Inclination, dtype: object

In [42]:
inclinations_in_degrees = full_table['Inclination'].str.replace('[^0-9.]', '')
inclinations_in_degrees = pd.to_numeric(inclinations_in_degrees, errors='coerce')
inclinations_in_degrees.head()

0    50.7
1    50.7
2     NaN
3    44.7
4    46.3
Name: Inclination, dtype: float64

In [43]:
full_table['Inclination'] = inclinations_in_degrees

### Longitude

In [44]:
full_table['Longitude'][81:86]

81    32.5° E
82     129° E
83        NaN
84        NaN
85        NaN
Name: Longitude, dtype: object

### Eccentricity

In [45]:
full_table['Eccentricity'].head()

0    0.00308
1    0.00325
2        NaN
3    0.04389
4    0.01735
Name: Eccentricity, dtype: object

### Epoch Start

In [46]:
full_table['Epoch Start'].head()

0    19 April 1975, 01:30:00 IST
1      7 June 1979, 01:30:00 IST
2                            NaN
3      18 July 1980, 1:30:00 IST
4       31 May 1981, 1:30:00 IST
Name: Epoch Start, dtype: object

In [47]:
epoch_date = full_table['Epoch Start'].str.extract('(\d+ [A-Za-z]+ \d{4})')[0]
epoch_date = pd.to_datetime(launch_date, errors='coerce')

epoch_time = full_table['Epoch Start'].str.extract('(\d{1,2}\:\d{1,2}(\:\d{1,2})?)')[0]

epoch_time_zone = full_table['Epoch Start'].str.extract('([A-Z]{3})')[0]

In [48]:
epoch_date.head()

0   1975-04-19
1   1979-06-07
2   1979-08-10
3   1980-07-18
4   1981-05-31
Name: 0, dtype: datetime64[ns]

In [49]:
epoch_time.head()

0    01:30:00
1    01:30:00
2         NaN
3     1:30:00
4     1:30:00
Name: 0, dtype: object

In [50]:
epoch_time_zone.head()

0    IST
1    IST
2    NaN
3    IST
4    IST
Name: 0, dtype: object

In [51]:
full_table['Epoch Start Date'] = epoch_date
full_table['Epoch Start Time'] = epoch_time
full_table['Epoch Start Timezone'] = epoch_time_zone

### Decay Date

In [52]:
full_table['Decay Date'].head()

0    11 February 1992
1    17 February 1989
2                 NaN
3         20 May 1981
4         8 June 1981
Name: Decay Date, dtype: object

In [53]:
decay_date = pd.to_datetime(full_table['Decay Date'], errors='coerce')
decay_date.head()

0   1992-02-11
1   1989-02-17
2          NaT
3   1981-05-20
4   1981-06-08
Name: Decay Date, dtype: datetime64[ns]

In [54]:
full_table['Decay Date'] = decay_date

### Launch Vehicle

In [55]:
full_table['Launch Vehicle'].head()

0                                 Interkosmos-II
1    Modified SS-5 (SKean IRBM) plus Upper Stage
2                                       SLV-3-E1
3                                       SLV-3-E2
4                                       SLV-3-D1
Name: Launch Vehicle, dtype: object

In [56]:
launch_vehicle_type = full_table['Launch Vehicle'].str.split('-| ').str[0]

In [57]:
launch_vehicle_type.head()

0    Interkosmos
1       Modified
2            SLV
3            SLV
4            SLV
Name: Launch Vehicle, dtype: object

In [58]:
launch_vehicle_type.value_counts()

PSLV           62
Ariane         25
GSLV           17
SLV             4
ASLV            4
Delta           2
Modified        2
Vostok          2
Molniya         1
Shuttle         1
SpaceX          1
Interkosmos     1
Name: Launch Vehicle, dtype: int64

In [59]:
full_table['Launch Vehicle Type'] = launch_vehicle_type

In [60]:
full_table.head()

,#,Name,Discipline,COSPAR ID,SatCat #,Launch Mass in kg,Dry Mass in kg,On-board Power in W,Launch Date,Launch Vehicle,...,Eccentricity,Epoch Start,Decay Date,Note,Launch Time,Launch Timezone,Epoch Start Date,Epoch Start Time,Epoch Start Timezone,Launch Vehicle Type
0,1,Aryabhatta,Earth Sciences Space Physics,1975-033A,07752,360.0,NaN,46.0,1975-04-19,Interkosmos-II,...,0.00308,"19 April 1975, 01:30:00 IST",1992-02-11,Active technological experience in building an...,13:10:00,IST,1975-04-19,01:30:00,IST,Interkosmos
1,2,Bhaskara Sega-I,Astronomy Communications Engineering Earth Sci...,1979-051A,11392,444.0,NaN,47.0,1979-06-07,Modified SS-5 (SKean IRBM) plus Upper Stage,...,0.00325,"7 June 1979, 01:30:00 IST",1989-02-17,First experimental remote sensing satellite. C...,16:00:00,IST,1979-06-07,01:30:00,IST,Modified
2,3,Rohini Technology Payload,Experimental,NaN,NaN,35.0,NaN,3.0,1979-08-10,SLV-3-E1,...,NaN,NaN,NaT,Intended for measuring in-flight performance o...,NaN,NaN,1979-08-10,NaN,NaN,SLV
3,4,Rohini RS-1 (Rohini-1B),Earth Sciences,1980-062A,11899,35.0,NaN,16.0,1980-07-18,SLV-3-E2,...,0.04389,"18 July 1980, 1:30:00 IST",1981-05-20,Used for measuring in-flight performance of se...,8:01:00,IST,1980-07-18,1:30:00,IST,SLV
4,5,Rohini RS-D1 (Rohini-2),Earth Sciences,1981-051A,12491,38.0,NaN,16.0,1981-05-31,SLV-3-D1,...,0.01735,"31 May 1981, 1:30:00 IST",1981-06-08,Used for conducting some remote sensing techno...,10:30:00,IST,1981-05-31,1:30:00,IST,SLV


In [61]:
full_table.tail()

,#,Name,Discipline,COSPAR ID,SatCat #,Launch Mass in kg,Dry Mass in kg,On-board Power in W,Launch Date,Launch Vehicle,...,Eccentricity,Epoch Start,Decay Date,Note,Launch Time,Launch Timezone,Epoch Start Date,Epoch Start Time,Epoch Start Timezone,Launch Vehicle Type
117,118,Orbiter of Chandrayaan-2,Lunar exploration,2019-042A,44441,2379.0,682.0,1.0,2019-07-22,GSLV Mk III M01,...,NaN,"20 August 2019, 09:02 IST (03:32 UTC)",NaT,India's second lunar exploration mission. Orbi...,14:43:12,IST,2019-07-22,09:02,IST,GSLV
118,119,Cartosat-3,Earth observation,2019-081A,44804,1625.0,NaN,2000.0,2019-11-27,PSLV-XL C47,...,NaN,NaN,NaT,13 American nano-satellites to be piggybacked ...,9:28:00,IST,2019-11-27,NaN,NaN,PSLV
119,120,RISAT-2BR1,Radar imaging,2019-089F,44857,628.0,NaN,NaN,2019-12-11,PSLV-QL C48,...,NaN,NaN,NaT,Has an improved resolution of 0.35 meters.,09:55,UTC,2019-12-11,NaN,NaN,PSLV
120,121,GSAT-30,Communications,2020-005A,45026,3357.0,NaN,6000.0,2020-01-16,Ariane 5 ECA VA-251,...,NaN,NaN,NaT,Replacement of INSAT-4A,21:05,UTC,2020-01-16,NaN,NaN,Ariane
121,TBD,GISAT-1,Earth observation,TBD,TBD,2268.0,NaN,2280.0,NaT,GSLV Mk II F10,...,NaN,NaN,NaT,India's first geostationary earth imaging sate...,NaN,NaN,NaT,NaN,NaN,GSLV


In [62]:
full_table.to_csv('clean.csv', index=False)